# Keras Tuner

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import he_uniform
import keras_tuner as kt

import credit_data
import visualkeras

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
train_data, test_data, train_label, test_label = credit_data.load_data()

train_data = train_data.todense()
test_data = test_data.todense()
train_label = utils.to_categorical(train_label)
test_label = utils.to_categorical(test_label)

print(train_data.shape, test_data.shape, train_label.shape, test_label.shape)

(15906, 67) (6818, 67) (15906, 3) (6818, 3)


In [3]:
def build_hyper_model(hp):
    model = keras.Sequential()

    hp_units = hp.Int('units_input', min_value=32, max_value=512, step=32)
    hp_activations = hp.Choice('activation_input', values=['relu', 'elu'])
    model.add(layers.Dense(input_dim=67, units=hp_units, activation=hp_activations, kernel_initializer=he_uniform()))
    model.add(layers.BatchNormalization())

    for layer_num in range(hp.Int('num_layers', min_value=1, max_value=5)): 
        hp_units = hp.Int('units_' + str(layer_num), min_value=32, max_value=512, step=32)
        hp_activations = hp.Choice('activation_' + str(layer_num), values=['relu', 'elu'])
        model.add(layers.Dense(hp_units, activation=hp_activations, kernel_initializer=he_uniform()))
        hp_dropouts = hp.Float('dropout_' + str(layer_num), 0.1, 0.5, step=0.1)
        model.add(layers.Dropout(hp_dropouts))
        model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=3, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    model.compile(optimizer=Adam(hp_learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return model

In [4]:
tuner = kt.BayesianOptimization(build_hyper_model,
                                objective = 'val_accuracy',
                                max_trials = 1000,
                                directory = 'keras_tuner',
                                project_name = 'model_hyper')

tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project keras_tuner/model_hyper/oracle.json
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

INFO:tensorflow:Reloading Tuner from keras_tuner/model_hyper/tuner0.json
Search space summary
Default search space size: 16
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_input (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu'], 'ordered': False}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu'], 'ordered': False}
dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'de

In [5]:
tuner.search(train_data, train_label, batch_size=100, epochs=10, validation_data=(test_data, test_label))

Trial 24 Complete [00h 01m 24s]
val_accuracy: 0.6629510521888733

Best val_accuracy So Far: 0.6911118030548096
Total elapsed time: 00h 36m 40s

Search: Running Trial #25

Value             |Best Value So Far |Hyperparameter
192               |512               |units_input
elu               |elu               |activation_input
5                 |2                 |num_layers
32                |32                |units_0
relu              |relu              |activation_0
0.4               |0                 |dropout_0
0.01              |0.01              |learning_rate
512               |512               |units_1
elu               |elu               |activation_1
0.3               |0.5               |dropout_1
448               |288               |units_2
elu               |elu               |activation_2
0.4               |0.4               |dropout_2
224               |32                |units_3
elu               |elu               |activation_3
0                 |0                 |

KeyboardInterrupt: 

In [ ]:
# 5) Check the result 

tuner.results_summary(num_trials=3) # Show "n" best trial results

Results summary
Results in test_prac_dir/MNIST_hyper_1
Showing 3 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 1
units_0: 512
activation_0: relu
learning_rate: 0.0001
units_1: 512
activation_1: relu
units_2: 32
activation_2: relu
Score: 0.9772999882698059
Trial summary
Hyperparameters:
num_layers: 3
units_0: 512
activation_0: elu
learning_rate: 0.0001
units_1: 512
activation_1: elu
units_2: 32
activation_2: relu
Score: 0.9736999869346619
Trial summary
Hyperparameters:
num_layers: 3
units_0: 512
activation_0: elu
learning_rate: 0.0001
units_1: 32
activation_1: relu
units_2: 32
activation_2: relu
Score: 0.9715999960899353


In [ ]:
# Check top-3 trials' hyper-params

top3_models = tuner.get_best_hyperparameters(num_trials=3)
# print(tuner.get_best_hyperparameters(num_trials=3)[0].space) # 특정 Trial의 Search-space 를 확인할 수 있음
# print(tuner.get_best_hyperparameters(num_trials=3)[0].values) # 특정 Trial에 적용된 Hyper-params를 확인할 수 있음

for idx, model in enumerate(top3_models):
    print('Model performance rank :', idx)
    print(model.values)
    print()


# Check the best trial's hyper-params

best_hps = top3_models[0]

print("""
The hyperparameter search is complete. 
* Optimal # of layers : {}
* Optimal value of the learning-rate : {}""".format(best_hps.get('num_layers'), best_hps.get('learning_rate')))

for layer_num in range(best_hps.get('num_layers')):
    print('Layer {} - # of Perceptrons :'.format(layer_num), best_hps.get('units_' + str(layer_num)))
    print('Layer {} - Applied activation function :'.format(layer_num), best_hps.get('activation_' + str(layer_num)))

Model performance rank : 0
{'num_layers': 1, 'units_0': 512, 'activation_0': 'relu', 'learning_rate': 0.0001, 'units_1': 512, 'activation_1': 'relu', 'units_2': 32, 'activation_2': 'relu'}

Model performance rank : 1
{'num_layers': 3, 'units_0': 512, 'activation_0': 'elu', 'learning_rate': 0.0001, 'units_1': 512, 'activation_1': 'elu', 'units_2': 32, 'activation_2': 'relu'}

Model performance rank : 2
{'num_layers': 3, 'units_0': 512, 'activation_0': 'elu', 'learning_rate': 0.0001, 'units_1': 32, 'activation_1': 'relu', 'units_2': 32, 'activation_2': 'relu'}


The hyperparameter search is complete. 
* Optimal # of layers : 1
* Optimal value of the learning-rate : 0.0001
Layer 0 - # of Perceptrons : 512
Layer 0 - Applied activation function : relu


In [ ]:
# Get the best model from trials

models = tuner.get_best_models(num_models=3) # Keras Sequential models
top_model = models[0]
top_model.summary()
print()

results = top_model.evaluate(x_test, y_test)
print('Cross-entropy :', results[0])
print('Accuracy :', results[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________

313/313 [==============================] - 1s 3ms/step - loss: 0.0770 - accuracy: 0.9773
Cross-entropy : 0.07701890170574188
Accuracy : 0.9772999882698059


In [ ]:
# We can retrain the model with the optimal hyperparameters from the search.
best_hps = top3_models[0]

# Build the model with the optimal hyperparameters and train it on the data.
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

results = model.evaluate(x_test, y_test)
print('Cross-entropy :', results[0])
print('Accuracy :', results[1])

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4527 - accuracy: 0.8856 - val_loss: 0.2434 - val_accuracy: 0.9318
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2154 - accuracy: 0.9410 - val_loss: 0.1819 - val_accuracy: 0.9477
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1624 - accuracy: 0.9550 - val_loss: 0.1454 - val_accuracy: 0.9579
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1301 - accuracy: 0.9643 - val_loss: 0.1224 - val_accuracy: 0.9649
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1077 - accuracy: 0.9705 - val_loss: 0.1065 - val_accuracy: 0.9694
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0908 - accuracy: 0.9755 - val_loss: 0.0984 - val_accuracy: 0.9713
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0778 - accuracy: 0.9792 - val_loss: 0.0884 - val_accuracy:

In [ ]:
# We can also find detailed logs, checkpoints, etc, in the folder "directory/project_name".

# The [test_prac_dir/MNIST_hyper_1] directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. 
# If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. 
# To disable this behavior, pass an additional [overwrite = True] argument while instantiating the tuner.

for trial in tuner.oracle.get_best_trials(num_trials=3):
    print('Trial-score is :', trial.score)
    print('Trial-directory(trial_id) is :', trial.trial_id)
    print()

# tuner.oracle.trials -> get all trial_id 

Trial-score is : 0.9772999882698059
Trial-directory(trial_id) is : ae3157b4a53b4b81dbf4907fb9a81d78

Trial-score is : 0.9736999869346619
Trial-directory(trial_id) is : 4176b5ee8939bdf6add6abc59d7b1bfa

Trial-score is : 0.9715999960899353
Trial-directory(trial_id) is : 6c4fd68fcba615c0504886a18afbe0f9



In [ ]:
top_model = models.load_model('keras_tuner/top_model.h5')
top_model.fit(train_data, train_label, batch_size=100, epochs=10, validation_data=(test_data, test_label))

results = top_model.evaluate(test_data, test_label)
print('Cross-entropy :', results[0])
print('Accuracy :', results[1])

Epoch 1/10
160/160 [==============================] - 8s 27ms/step - loss: 0.7800 - accuracy: 0.6900 - val_loss: 0.8296 - val_accuracy: 0.6789
Epoch 2/10
160/160 [==============================] - 4s 27ms/step - loss: 0.7669 - accuracy: 0.6936 - val_loss: 0.8226 - val_accuracy: 0.6888
Epoch 3/10
160/160 [==============================] - 3s 21ms/step - loss: 0.7539 - accuracy: 0.7006 - val_loss: 0.8272 - val_accuracy: 0.6760
Epoch 4/10
160/160 [==============================] - 3s 20ms/step - loss: 0.7489 - accuracy: 0.7007 - val_loss: 0.8252 - val_accuracy: 0.6797
Epoch 5/10
160/160 [==============================] - 3s 19ms/step - loss: 0.7411 - accuracy: 0.7030 - val_loss: 0.8277 - val_accuracy: 0.6866
Epoch 6/10
160/160 [==============================] - 3s 21ms/step - loss: 0.7345 - accuracy: 0.7051 - val_loss: 0.8349 - val_accuracy: 0.6806
Epoch 7/10
160/160 [==============================] - 3s 20ms/step - loss: 0.7222 - accuracy: 0.7114 - val_loss: 0.8388 - val_accuracy: 0.6773

<br> 

## (Appendix) Use pre-trained models for computer vision: HyperResNet & HyperXception

* pre-compiled with loss='categorical_crossentropy' & metrics=\['accuracy']
* Next model-trainings take too much time, so try it if you are needed.

In [ ]:
from kerastuner.applications import HyperResNet, HyperXception
from kerastuner.tuners import Hyperband
from kerastuner import HyperParameters

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# (img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data() # if you want to use Fashion-MNIST instead of MNIST

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Prepare the data as needed
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

### Pre-trained ResNet

In [ ]:
hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

# Hyperband performs better than random search with low level computing resource. (Hyperband @ https://arxiv.org/abs/1603.06560)
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=40,
    directory='test_prac_dir',
    project_name='MNIST_Resnet_1')

tuner.search(x_train, y_train, epochs=20, validation_data=(x_test, y_test))
tuner.results_summary()

### Pre-trained Xception

In [ ]:
hypermodel = HyperXception(input_shape=(28, 28, 1), classes=10)

# This will override the `learning_rate` parameter with your own selection of choices
hp = HyperParameters()
hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

# we can easily restrict the search space to just a few parameters
tuner = Hyperband(
    hypermodel,
    hyperparameters=hp, # ADDED
    tune_new_entries=False, # ADDED (`tune_new_entries=False` prevents unlisted parameters from being tuned)
    objective='val_accuracy',
    max_epochs=40,
    directory='test_prac_dir',
    project_name='MNIST_Xception_1')

tuner.search(x_train, y_train, epochs=20, validation_data=(x_test, y_test))
tuner.results_summary()

# What if you want to tune all available parameters in a hypermodel except the learning rate? @ https://j.mp/2J7jzHj

### Changing the existing optimizer, loss, or metrics

In [ ]:
hypermodel = HyperXception(input_shape=(28, 28, 1), classes=10)

tuner = Hyperband(
    hypermodel,
    optimizer=keras.optimizers.Adam(1e-3),
    loss='mse',
    metrics=[keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall')],
    objective='mse',
    max_epochs=40,
    directory='test_prac_dir',
    project_name='MNIST_Xception_1')

tuner.search(x_train, y_train, epochs=20, validation_data=(x_test, y_test))
tuner.results_summary()